In [1]:
from os import listdir

import pandas as pd

In [2]:
pipeline_dir = '/Users/pti/challenges/4761_fair_fairytale/fair-fairytale-nlp/data/pipeline/'

In [3]:
all_story_ids = listdir(pipeline_dir)

In [4]:
len(all_story_ids)

283

In [5]:
all_story_ids.remove('test')
all_story_ids.remove('problems')
all_story_ids.remove('old')
all_story_ids.remove('.DS_Store')
all_story_ids.remove('sentences')

In [6]:
len(all_story_ids)

278

In [7]:
all_story_ids[0]

'the-brownie-of-the-lake'

In [8]:
all_story_ids[-1]

'the-little-hunting-dog'

In [9]:
dfs = []
problem_stories = []
sentences = {}

for story_id in all_story_ids:
    try:
        df = pd.read_csv(pipeline_dir + story_id + '/' + story_id + '.actions_args.csv')
        df.to_csv(pipeline_dir + story_id + '/' + story_id + '.actions_args.old.csv', index = False)
        dfs.append(df)
        sentences[story_id] = pd.read_csv(pipeline_dir + story_id + '/' + story_id + '.sentences.csv')['text'].tolist()
    except FileNotFoundError:
        problem_stories.append(story_id)

In [10]:
len(dfs)

278

In [11]:
len(sentences)

278

In [12]:
for i, df in enumerate(dfs):
    new_dobj_coref_ids = []
    story_id = all_story_ids[i]
    # Clean up column names
    if 'supersense_category_x' in df.columns and 'supersense_category_y' in df.columns and 'supersense_category' not in df.columns:
        df.drop(columns = ['supersense_category_y'], inplace = True)
        df.rename(columns = {'supersense_category_x': 'supersense_category'}, inplace = True)
    elif 'supersense_category_x' in df.columns and 'supersense_category_y' in df.columns and 'supersense_category' in df.columns:
        df.drop(columns = ['supersense_category_x', 'supersense_category_y'], inplace = True)
    for i, row in df.iterrows():
        subj_corefs = set(str(row['subj_coref_ids']).split(','))
        if 'nan' in subj_corefs:
            subj_corefs.remove('nan')
        dobj_corefs = set(str(row['dobj_coref_ids']).split(','))
        if 'nan' in dobj_corefs:
            dobj_corefs.remove('nan')
        overlaps = subj_corefs.intersection(dobj_corefs)
        if len(overlaps) > 0:
            # print(sentences[story_id][row['sentence_id']])
            # print(row['verb'])
            # print("subject coref: ", row['subj_coref_ids'])
            # print("subject: ", sentences[story_id][row['sentence_id']][int(row['subj_start_byte']):int(row['subj_end_byte'])])
            # print("direct_object coref: ", row['dobj_coref_ids'])
            # print("direct_object: ", sentences[story_id][row['sentence_id']][int(row['dobj_start_byte']):int(row['dobj_end_byte'])])
            overlaps = list(overlaps)[0]
            dobj_corefs.remove(overlaps)
            # print(','.join(dobj_corefs))
        new_dobj_coref_ids.append(','.join(list(dobj_corefs)))
    df.loc[:, ['dobj_coref_ids']] = new_dobj_coref_ids
    df.to_csv(pipeline_dir + story_id + '/' + story_id + '.actions_args.csv', index = False)